In [5]:
# Imports
# Use mostly numpy for script performance
import numpy as np
import pandas as pd

# 1. Load data
data = pd.read_csv('trackingData_v2.csv')

# 2. Filter out the necessary data
valid_data = data[data['ClockState'] == 'ClockStateRunning'].copy()

# 3. Calculate differences for consecutive rows for locations and time
valid_data['dx'] = valid_data.groupby('player')['Location.X'].diff()
valid_data['dy'] = valid_data.groupby('player')['Location.Y'].diff()
valid_data['dt'] = valid_data.groupby('player')['UTCTime'].diff()

# 4. Calculate distance, speed, and acceleration
valid_data['distance'] = np.sqrt(valid_data['dx']**2 + valid_data['dy']**2)
valid_data['speed'] = valid_data['distance'] / valid_data['dt']
valid_data['acceleration'] = valid_data.groupby('player')['speed'].diff() / valid_data['dt']

# 5. Convert OnPlayingSurface from TRUE/FALSE to 1/0
valid_data['OnPlayingSurface'] = valid_data['OnPlayingSurface'].astype(int)

# 6. Identifying shifts using diff
valid_data['shift_change'] = valid_data['OnPlayingSurface'].diff().fillna(0)
valid_data['start_of_shift'] = (valid_data['shift_change'] == 1).astype(int)
valid_data['shift_id'] = valid_data['start_of_shift'].cumsum()

# 7. Determine Statistics for Each Player
players_stats = {}
for player in valid_data['player'].unique():
    player_data = valid_data[valid_data['player'] == player]
    
    # Total distance
    total_distance = player_data['distance'].sum()
    
    # Average speed per shift
    average_speed_per_shift = player_data.groupby('shift_id')['speed'].mean()
    
    # Max speed and acceleration
    max_speed = player_data['speed'].max()
    max_acceleration = player_data['acceleration'].max()
    
    # Print player stats
    print(f"\nPlayer {player} stats:")
    print(f"Total Distance: {total_distance:.2f} feet")
    print(f"Max Speed: {max_speed:.2f} ft/s")
    print(f"Max Acceleration: {max_acceleration:.2f} ft/s^2")
    print(f"Average Speed per shift (ft/s):")
    for idx, avg_speed in enumerate(average_speed_per_shift.values, 1):
        print(f"Shift {idx}: {avg_speed:.2f}")
    
    players_stats[player] = {
        'avg_speed_per_shift': average_speed_per_shift.values,
        'max_speed': max_speed,
        'max_acceleration': max_acceleration,
        'total_distance': total_distance
    }

# 8. Rank players based on acceleration first and then speed
ranking = sorted(players_stats.keys(), 
                 key=lambda x: (players_stats[x]['max_acceleration'], players_stats[x]['max_speed']), reverse=True)

print(f"\nRanking from best to worst skater: {ranking}")



Player 4 stats:
Total Distance: 17415.92 feet
Max Speed: 32.36 ft/s
Max Acceleration: 96.73 ft/s^2
Average Speed per shift (ft/s):
Shift 1: 0.40
Shift 2: 12.88
Shift 3: 11.44
Shift 4: 12.69
Shift 5: 10.28
Shift 6: 8.78
Shift 7: 9.01
Shift 8: 7.65
Shift 9: 6.10
Shift 10: 9.34
Shift 11: 8.08
Shift 12: 12.09
Shift 13: 9.47
Shift 14: 12.75
Shift 15: 13.91
Shift 16: 9.35
Shift 17: 11.54
Shift 18: 13.00
Shift 19: 10.80
Shift 20: 10.85
Shift 21: 12.79
Shift 22: 10.08
Shift 23: 8.59
Shift 24: 6.92
Shift 25: 9.80

Player 1 stats:
Total Distance: 10619.52 feet
Max Speed: 29.73 ft/s
Max Acceleration: 161.96 ft/s^2
Average Speed per shift (ft/s):
Shift 1: 0.19
Shift 2: 9.70
Shift 3: 11.45
Shift 4: 14.15
Shift 5: 7.59
Shift 6: 10.81
Shift 7: 7.36
Shift 8: 10.67
Shift 9: 8.96
Shift 10: 5.24
Shift 11: 10.04
Shift 12: 10.13
Shift 13: 9.71
Shift 14: 4.44
Shift 15: 9.42

Player 2 stats:
Total Distance: 12821.66 feet
Max Speed: 29.29 ft/s
Max Acceleration: 92.02 ft/s^2
Average Speed per shift (ft/s):
Sh